In [1]:
import numpy as np
import mxnet as mx
from mxnet import nd
from tqdm import tqdm

# Image Preprocess 

- size: > 224 
- normalized: mean = [0.485, 0.456, 0.406],  std = [0.229, 0.224, 0.225]
- shape:  RGB images of shape (N x 3 x H x W)

https://mxnet.incubator.apache.org/api/python/gluon/model_zoo.html#mxnet.gluon.model_zoo

In [2]:
preprocess_list = [                 #建立预处理图片内容列表，也就是对图片要做的事情  用于下面会被使用    我感觉这里会对图片进行处理，因为我看到下面有讲图像旋转 设置均值和方差等
    lambda img: img.astype("float32")/255,  #对图片特征进行处理并  进行归到0-1内的数值
    mx.image.ForceResizeAug((224, 224)),  #调整大小 
    mx.image.ColorNormalizeAug(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225]),  # 对图片进行均值和  方差规范化
    lambda img: nd.transpose(img,(2,0,1))# j进行图片的 翻转，官方上这是一种图片矩阵的转变，而不是那种角度转变那种吧。
]
#  定义图片预处理函数，会调用上面的结构将图片变成上面的形式。进行预处理
def image_preprocess(img):
    for f in preprocess_list:  #这里的意思是对上面list列表中的每一个操作一个个的执行，去对图片进行处理
        img = f(img)
    return img

# Data Loader

In [3]:
def transform(img, label):       #指定图形的转换
    return image_preprocess(img), label

def load_data(data_dir, load_batch_size = 32, f_trans=transform):   #使用gluon方法去 生成数据    生成对序列数据加载后的形式  
    imgs = mx.gluon.data.vision.ImageFolderDataset(data_dir, transform=transform)#用于加载文件夹结构中图像文件的数据集  会对这里面的所有图片执行transform函数
    data = mx.gluon.data.DataLoader(imgs, load_batch_size, last_batch="keep")# z这里为进行预训练提取特征做准备，指定每次提取的数量。
    return data

In [4]:
train_data = load_data("D:\dog_data\dogbreed\\train_gy")   #这里会按批次加载数据进行处理   处理之后会进一步的投入网络中去
test_data = load_data("D:\dog_data\dogbreed\\test_gy")

# Pretrained Net Loader

In [5]:
ctx = mx.cpu()                #这里使用gluon框架会加载残差网络      使用残差网络进行模型的训练
resnet50_v2 = mx.gluon.model_zoo.vision.resnet50_v2(pretrained=True, ctx=ctx)

# Feature Extraction From Pretrained Net

In [6]:
def extract_features(net, data, ctx):    #定义提取特征的函数    我知道了 这里实际上应该有个跑网络的过程，  怪不得花了那么长时间。。从预训练模型里获得数据 
    rst_X, rst_y = [], []                #这里有三个主体， 使用 net网络，是ctx机器，对data数据进行跑，将残差网络从头跑到尾部，从里面获得输出的特征结果
    for X, y in tqdm(data):    #tqdm是一个定义的进度条，这里会 定义数据的批次提取特征的进度
        Xi = net.features(X.as_in_context(ctx))
        rst_X.append(Xi.asnumpy())
        rst_y.append(y.asnumpy())
    return np.concatenate(rst_X, axis=0), np.concatenate(rst_y, axis=0) 

In [7]:
X_train_resnet50_v2, y_train = extract_features(resnet50_v2, train_data, ctx)   #调用上面的提取函数从train_data数据中提取特征  每批次算一个百分比，10000/32=320

100%|████████████████████████████████████████| 320/320 [37:48<00:00,  7.09s/it]


In [ ]:
X_test_resnet50_v2, _ = extract_features(resnet50_v2, test_data, ctx)

 93%|█████████████████████████████████████▏  | 301/324 [38:26<02:56,  7.66s/it]

# Save features and labels for future use

In [61]:
import h5py              #使用  h5py将预训练残杀网络的数据保存起来    将模型和 对应的训练标签单独同意保持在h5文件中去。。。   将提取的所有特征和对应标签进行保存
with h5py.File('D:\dog_data\dogbreed/resnet50_v2_pretrained_Xy.h5', 'w') as f:
    f['X_train_resnet50_v2'] = X_train_resnet50_v2
    f['X_test_resnet50_v2'] = X_test_resnet50_v2
    f['y_train'] = y_train